In [3]:
import parse_tfl_json
import google

In [4]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime as sjt
where tfl_message = 'ok' and
 sjt.icscode not in (select icscode from tt_h.all_stations where in_sprawl = true)
"""

df = pd.read_sql(sql, conn)

In [5]:
from process_journeys import get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [6]:
for r in df.iterrows():
    index = r[0]
    row = r[1]

    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    

    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]

    # Keep only journeys that end in central London  
    try:
        journeys = remove_journeys_not_arriving_clondon(journeys)
    except:
        print("Problem with journey starting at {}".format(row["station_name"]))
        continue

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]

In [9]:
df3 = df2.sort_values(["station_name", "tfl_dest", "total_journeytime"]).drop_duplicates(["station_name", "tfl_dest"])

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
5534,3813,90000007,Aber,ok,E145HP,Aber Rail Station → Cardiff Central Rail Stati...,1.0,Paddington Station,187.0,51.574964,-3.229828,51.516993,-0.177388,35.668471,8.322643,222.668471
5531,3813,90000007,Aber,ok,SW1H9AJ,Aber Rail Station → Cardiff Central Rail Stati...,1.0,Paddington Station,187.0,51.574964,-3.229828,51.516993,-0.177388,11.964942,2.791820,198.964942
1103,3801,90000019,Abercynon,ok,E145HP,Abercynon North Rail Station → Abercynon Rail ...,2.0,Paddington Station,220.0,51.645312,-3.326803,51.516993,-0.177388,35.668471,8.322643,255.668471
1020,3801,90000019,Abercynon,ok,SW1H9AJ,Abercynon North Rail Station → Abercynon Rail ...,2.0,Paddington Station,220.0,51.645312,-3.326803,51.516993,-0.177388,11.964942,2.791820,231.964942
10416,3982,90000004,Aberdare,ok,E145HP,Aberdare Rail Station → Cardiff Central Rail S...,1.0,Paddington Station,248.0,51.715056,-3.443083,51.516993,-0.177388,35.668471,8.322643,283.668471
10413,3982,90000004,Aberdare,ok,SW1H9AJ,Aberdare Rail Station → Cardiff Central Rail S...,1.0,Paddington Station,248.0,51.715056,-3.443083,51.516993,-0.177388,11.964942,2.791820,259.964942
10708,8976,90000024,Aberdeen,ok,E145HP,Aberdeen Rail Station → London King's Cross Ra...,0.0,London King's Cross Rail Station,419.0,57.143703,-2.098685,51.531579,-0.123243,25.098950,5.856422,444.098950
10705,8976,90000024,Aberdeen,ok,SW1H9AJ,Aberdeen Rail Station → London King's Cross Ra...,0.0,London King's Cross Rail Station,419.0,57.143703,-2.098685,51.531579,-0.123243,19.895412,4.642263,438.895412
1122,9090,90000005,Aberdour,ok,E145HP,Aberdour Rail Station → Edinburgh Rail Station...,1.0,London King's Cross Rail Station,307.0,56.054590,-3.300560,51.531579,-0.123243,25.098950,5.856422,332.098950
1085,9090,90000005,Aberdour,ok,SW1H9AJ,Aberdour Rail Station → Edinburgh Rail Station...,1.0,London King's Cross Rail Station,307.0,56.054590,-3.300560,51.531579,-0.123243,19.895412,4.642263,326.895412


In [10]:
df_pf = df3[df3["tfl_dest"] == "SW1H9AJ"]
df_cw = df3[df3["tfl_dest"] == "E145HP"]

In [11]:
all_stations_data = df_pf.merge(df_cw, how='left', on=["nlc", "icscode", "station_name", "tfl_message", "depart_lat", "depart_lng"], suffixes=("_pf", "_cw"))
all_stations_data

,nlc,icscode,station_name,tfl_message,tfl_dest_pf,natrail_journey_summary_pf,natrail_train_changes_pf,final_arrival_pf,natrail_journey_minutes_pf,depart_lat,...,tfl_dest_cw,natrail_journey_summary_cw,natrail_train_changes_cw,final_arrival_cw,natrail_journey_minutes_cw,arrive_lat_cw,arrive_lng_cw,cycle_minutes_cw,cycle_miles_cw,total_journeytime_cw
0,3813,90000007,Aber,ok,SW1H9AJ,Aber Rail Station → Cardiff Central Rail Stati...,1.0,Paddington Station,187.0,51.574964,...,E145HP,Aber Rail Station → Cardiff Central Rail Stati...,1.0,Paddington Station,187.0,51.516993,-0.177388,35.668471,8.322643,222.668471
1,3801,90000019,Abercynon,ok,SW1H9AJ,Abercynon North Rail Station → Abercynon Rail ...,2.0,Paddington Station,220.0,51.645312,...,E145HP,Abercynon North Rail Station → Abercynon Rail ...,2.0,Paddington Station,220.0,51.516993,-0.177388,35.668471,8.322643,255.668471
2,3982,90000004,Aberdare,ok,SW1H9AJ,Aberdare Rail Station → Cardiff Central Rail S...,1.0,Paddington Station,248.0,51.715056,...,E145HP,Aberdare Rail Station → Cardiff Central Rail S...,1.0,Paddington Station,248.0,51.516993,-0.177388,35.668471,8.322643,283.668471
3,8976,90000024,Aberdeen,ok,SW1H9AJ,Aberdeen Rail Station → London King's Cross Ra...,0.0,London King's Cross Rail Station,419.0,57.143703,...,E145HP,Aberdeen Rail Station → London King's Cross Ra...,0.0,London King's Cross Rail Station,419.0,51.531579,-0.123243,25.098950,5.856422,444.098950
4,9090,90000005,Aberdour,ok,SW1H9AJ,Aberdour Rail Station → Edinburgh Rail Station...,1.0,London King's Cross Rail Station,307.0,56.054590,...,E145HP,Aberdour Rail Station → Edinburgh Rail Station...,1.0,London King's Cross Rail Station,307.0,51.531579,-0.123243,25.098950,5.856422,332.098950
5,4435,90000006,Aberdovey,ok,SW1H9AJ,Aberdovey Rail Station → Wolverhampton Rail St...,2.0,Euston Station,263.0,52.543955,...,E145HP,Aberdovey Rail Station → Wolverhampton Rail St...,2.0,Euston Station,263.0,51.528952,-0.135000,27.961695,6.524395,290.961695
6,4440,90000033,Abererch,ok,SW1H9AJ,Abererch Rail Station → Shrewsbury Rail Statio...,1.0,Euston Station,376.0,52.898581,...,E145HP,Abererch Rail Station → Shrewsbury Rail Statio...,1.0,Euston Station,376.0,51.528952,-0.135000,27.961695,6.524395,403.961695
7,3602,90000035,Abergavenny,ok,SW1H9AJ,Abergavenny Rail Station → Newport (S Wales) R...,1.0,Paddington Station,177.0,51.816688,...,E145HP,Abergavenny Rail Station → Newport (S Wales) R...,1.0,Paddington Station,177.0,51.516993,-0.177388,35.668471,8.322643,212.668471
8,2448,90000008,Abergele & Pensarn,ok,SW1H9AJ,Abergele & Pensarn Rail Station → Chester Rail...,2.0,Euston Station,175.0,53.294571,...,E145HP,Abergele & Pensarn Rail Station → Chester Rail...,2.0,Euston Station,175.0,51.528952,-0.135000,27.961695,6.524395,202.961695
9,4303,90000040,Aberystwyth,ok,SW1H9AJ,Aberystwyth Rail Station → Shrewsbury Rail Sta...,3.0,Euston Station,269.0,52.414043,...,E145HP,Aberystwyth Rail Station → Shrewsbury Rail Sta...,3.0,Euston Station,269.0,51.528952,-0.135000,27.961695,6.524395,296.961695


In [12]:
# Write out to Postgres
all_stations_data.to_sql("stations_maps_data", engine, schema="tt_h", if_exists="replace", index=False)

In [13]:
all_stations_data.to_csv("stations_maps_data.csv", encoding='utf-8')